In [46]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [47]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/validation'

In [48]:
def dataframe(add):
    labels = []
    images = []
    for x in os.listdir(add):
        if os.path.isdir(os.path.join(add, x)):
            j = os.path.join(add, x)
            for js in os.listdir(j):
                if os.path.isfile(os.path.join(add,x,js)):
                    labels.append(x)
                    images.append(os.path.join(add,x,js))
    return labels,images


In [61]:
train = pd.DataFrame()
train['label'],train['image']= dataframe(TRAIN_DIR)

In [62]:
print(train)

         label                           image
0        happy     images/train/happy/3578.jpg
1        happy    images/train/happy/16988.jpg
2        happy     images/train/happy/2666.jpg
3        happy     images/train/happy/5109.jpg
4        happy    images/train/happy/11981.jpg
...        ...                             ...
28816  disgust  images/train/disgust/10112.jpg
28817  disgust  images/train/disgust/21668.jpg
28818  disgust   images/train/disgust/7049.jpg
28819  disgust   images/train/disgust/9716.jpg
28820  disgust   images/train/disgust/3561.jpg

[28821 rows x 2 columns]


In [63]:
test = pd.DataFrame()
test['label'],test['image'] = dataframe(TEST_DIR)

In [64]:
print(test)
print(test['image'])

        label                                image
0       happy    images/validation/happy/23933.jpg
1       happy    images/validation/happy/24906.jpg
2       happy    images/validation/happy/18033.jpg
3       happy    images/validation/happy/15271.jpg
4       happy    images/validation/happy/26888.jpg
...       ...                                  ...
7061  disgust  images/validation/disgust/20761.jpg
7062  disgust  images/validation/disgust/28710.jpg
7063  disgust  images/validation/disgust/23876.jpg
7064  disgust   images/validation/disgust/9460.jpg
7065  disgust  images/validation/disgust/35580.jpg

[7066 rows x 2 columns]
0         images/validation/happy/23933.jpg
1         images/validation/happy/24906.jpg
2         images/validation/happy/18033.jpg
3         images/validation/happy/15271.jpg
4         images/validation/happy/26888.jpg
                       ...                 
7061    images/validation/disgust/20761.jpg
7062    images/validation/disgust/28710.jpg
7063    ima

In [58]:
from tqdm.notebook import tqdm

In [65]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [66]:
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

/Users/shyampatel/anaconda3/lib/python3.11/site-packages/keras/src/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [67]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [68]:
x_train = train_features/255.0
x_test = test_features/255.0

In [69]:
from sklearn.preprocessing import LabelEncoder

In [70]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [71]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [72]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [73]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [74]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [75]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
226/226 [==============================] - 177s 782ms/step - loss: 1.8227 - accuracy: 0.2421 - val_loss: 1.8064 - val_accuracy: 0.2637
Epoch 2/100
226/226 [==============================] - 192s 849ms/step - loss: 1.7890 - accuracy: 0.2560 - val_loss: 1.6980 - val_accuracy: 0.2985
Epoch 3/100
226/226 [==============================] - 371s 2s/step - loss: 1.6953 - accuracy: 0.3090 - val_loss: 1.6456 - val_accuracy: 0.3351
Epoch 4/100
226/226 [==============================] - 185s 820ms/step - loss: 1.5686 - accuracy: 0.3825 - val_loss: 1.4860 - val_accuracy: 0.4247
Epoch 5/100
226/226 [==============================] - 183s 808ms/step - loss: 1.4747 - accuracy: 0.4290 - val_loss: 1.3458 - val_accuracy: 0.4877
Epoch 6/100
226/226 [==============================] - 184s 815ms/step - loss: 1.4104 - accuracy: 0.4534 - val_loss: 1.3398 - val_accuracy: 0.4919
Epoch 7/100
226/226 [==============================] - 180s 798ms/step - loss: 1.3813 - accuracy: 0.4705 - val_loss: 1.26

Epoch 57/100
226/226 [==============================] - 253s 1s/step - loss: 0.9238 - accuracy: 0.6572 - val_loss: 1.0362 - val_accuracy: 0.6219
Epoch 58/100
226/226 [==============================] - 263s 1s/step - loss: 0.9256 - accuracy: 0.6534 - val_loss: 1.0316 - val_accuracy: 0.6233
Epoch 59/100
226/226 [==============================] - 260s 1s/step - loss: 0.9201 - accuracy: 0.6565 - val_loss: 1.0395 - val_accuracy: 0.6193
Epoch 60/100
226/226 [==============================] - 263s 1s/step - loss: 0.9093 - accuracy: 0.6626 - val_loss: 1.0198 - val_accuracy: 0.6288
Epoch 61/100
226/226 [==============================] - 260s 1s/step - loss: 0.9122 - accuracy: 0.6587 - val_loss: 1.0246 - val_accuracy: 0.6279
Epoch 62/100
226/226 [==============================] - 260s 1s/step - loss: 0.8971 - accuracy: 0.6674 - val_loss: 1.0357 - val_accuracy: 0.6258
Epoch 63/100
226/226 [==============================] - 261s 1s/step - loss: 0.9065 - accuracy: 0.6639 - val_loss: 1.0279 - val_ac

In [76]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

/Users/shyampatel/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [77]:
from keras.models import model_from_json

In [80]:
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [81]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [82]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [110]:
# pred_labels = []
# for imgs in test['image']:
#     pred_labels.append(imgs)
#     img = ef(imgs)
#     pred = model.predict(img)
#     pred_label = label[pred.argmax()]
#     imgs = ef(imgs)
#     pred = model.predict(img) 
#     pred_labels.append(label[pred.argmax()])
# print(len(pred_labels))
pred_labels = []
# print(test['image'][7005])
for i in range(0,len(test['image'])):
    img = ef(test['image'][i])
    pred = model.predict(img)
    pred_label = label[pred.argmax()]
    pred_labels.append(pred_label)

# print(pred_labels)
    

1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step


...

1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 11ms/step


In [113]:
acc = 0
for i in range(0,len(pred_labels)):
    if pred_labels[i]==test['label'][i]:
        acc = acc+1
print((acc/len(pred_labels))*100)

63.33144636286442


## Changes made

In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a Sequential model
model_r = Sequential()

# Convolutional layers
model_r.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model_r.add(BatchNormalization())
model_r.add(MaxPooling2D(pool_size=(2, 2)))
model_r.add(Dropout(0.4))

model_r.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model_r.add(BatchNormalization())
model_r.add(MaxPooling2D(pool_size=(2, 2)))
model_r.add(Dropout(0.4))

model_r.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model_r.add(BatchNormalization())
model_r.add(MaxPooling2D(pool_size=(2, 2)))
model_r.add(Dropout(0.4))

model_r.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model_r.add(BatchNormalization())
model_r.add(MaxPooling2D(pool_size=(2, 2)))
model_r.add(Dropout(0.4))

model_r.add(Flatten())

# Fully connected layers
model_r.add(Dense(512, activation='relu'))
model_r.add(BatchNormalization())
model_r.add(Dropout(0.4))

model_r.add(Dense(256, activation='relu'))
model_r.add(BatchNormalization())
model_r.add(Dropout(0.3))

# Output layer
model_r.add(Dense(7, activation='softmax'))

# Compile the model with Adam optimizer and categorical crossentropy loss
model_r.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model_r.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 46, 46, 128)       1280      
                                                                 
 batch_normalization_6 (Bat  (None, 46, 46, 128)       512       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 23, 23, 128)       0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 23, 23, 128)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 batch_normalization_7 (Bat  (None, 21, 21, 256)      

In [ ]:
model_r.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
226/226 [==============================] - 213s 937ms/step - loss: 2.1167 - accuracy: 0.2254 - val_loss: 2.0693 - val_accuracy: 0.2583
Epoch 2/100
226/226 [==============================] - 1116s 5s/step - loss: 1.6722 - accuracy: 0.3625 - val_loss: 2.0627 - val_accuracy: 0.2958
Epoch 3/100
226/226 [==============================] - 525s 2s/step - loss: 1.4607 - accuracy: 0.4435 - val_loss: 1.6870 - val_accuracy: 0.3566
Epoch 4/100
226/226 [==============================] - 269s 1s/step - loss: 1.3480 - accuracy: 0.4840 - val_loss: 1.3101 - val_accuracy: 0.4955
Epoch 5/100
226/226 [==============================] - 264s 1s/step - loss: 1.2718 - accuracy: 0.5136 - val_loss: 1.1860 - val_accuracy: 0.5453
Epoch 6/100
226/226 [==============================] - 239s 1s/step - loss: 1.2161 - accuracy: 0.5361 - val_loss: 1.3180 - val_accuracy: 0.5116
Epoch 7/100
226/226 [==============================] - 238s 1s/step - loss: 1.1674 - accuracy: 0.5574 - val_loss: 1.2571 - val_accur